---
# Exercício Titanic - kNN, Regressão Logística
---
**Machine Learning em Projetos (Erick Muzart e Fernando melo)**   
Tópicos:
- One-hot Encoder get-dummies
- Knn
- Regressão logística
- Matriz de confusão

<br>
------------------------------------------------
   
Usando dados do Titanic, descritos em: https://www.openml.org/d/40945

## Carregar bibliotecas

In [45]:
# Importar bibliotecas pandas, plot_confusion_matrix, train_test_split, matplotlib, metrics
import pandas as pd

## Análise exploratória dos dados

In [46]:
# Ler dados do Titanic a partir do site Openml.org
df = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl', na_values='?')
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [47]:
nan_series = df.isna().sum()/len(df)
nan_series

pclass       0.000000
survived     0.000000
name         0.000000
sex          0.000000
age          0.200917
sibsp        0.000000
parch        0.000000
ticket       0.000000
fare         0.000764
cabin        0.774637
embarked     0.001528
boat         0.628724
body         0.907563
home.dest    0.430863
dtype: float64

In [48]:
highest_nans = nan_series[nan_series > 0.5].index
highest_nans

Index(['cabin', 'boat', 'body'], dtype='object')

In [49]:
df = df.drop(highest_nans, axis=1)
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,S,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,S,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,C,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,C,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,C,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,C,NaN


In [50]:
nan_series = df.isna().sum()/len(df)
nan_series[nan_series > 0]

age          0.200917
fare         0.000764
embarked     0.001528
home.dest    0.430863
dtype: float64

In [51]:
df['home.dest'].unique()

array(['St Louis, MO', 'Montreal, PQ / Chesterville, ON', 'New York, NY',
       'Hudson, NY', 'Belfast, NI', 'Bayside, Queens, NY',
       'Montevideo, Uruguay', 'Paris, France', nan, 'Hessle, Yorks',
       'Montreal, PQ', 'Winnipeg, MN', 'San Francisco, CA',
       'Dowagiac, MI', 'Stockholm, Sweden / Washington, DC',
       'Trenton, NJ', 'Glen Ridge, NJ', 'Youngstown, OH',
       'Birkdale, England Cleveland, Ohio', 'London / Winnipeg, MB',
       'Cooperstown, NY', 'St Leonards-on-Sea, England Ohio',
       'Los Angeles, CA', 'Pomeroy, WA', 'Omaha, NE', 'Philadelphia, PA',
       'Denver, CO', 'Belmont, MA', 'Washington, DC',
       'Austria-Hungary / Germantown, Philadelphia, PA',
       'Germantown, Philadelphia, PA', 'Bryn Mawr, PA',
       'Ascot, Berkshire / Rochester, NY', 'Little Onn Hall, Staffs',
       'Amenia, ND', 'New York, NY / Ithaca, NY', 'London, England',
       'Liverpool, England / Belfast', 'Stoughton, MA', 'Victoria, BC',
       'Lakewood, NJ', 'Roachdale, I

In [52]:
df = df.drop('home.dest', axis=1)
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,S
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,S
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,S
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,S
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,C
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,C
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,C
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,C


In [53]:
nan_series = df.isna().sum()/len(df)
nan_series = nan_series[nan_series > 0]
nan_series

age         0.200917
fare        0.000764
embarked    0.001528
dtype: float64

In [54]:
df.embarked.unique()

array(['S', 'C', nan, 'Q'], dtype=object)

In [56]:
df.embarked.fillna(df.embarked.mode()[0], inplace=True)
df.embarked.value_counts()

S    916
C    270
Q    123
Name: embarked, dtype: int64

In [57]:
nan_series = df.isna().sum()/len(df)
nan_series = nan_series[nan_series > 0]
nan_series

age     0.200917
fare    0.000764
dtype: float64

In [58]:
df[nan_series.index].describe()

,age,fare
count,1046.000000,1308.000000
mean,29.881135,33.295479
std,14.413500,51.758668
min,0.166700,0.000000
25%,21.000000,7.895800
50%,28.000000,14.454200
75%,39.000000,31.275000
max,80.000000,512.329200


In [59]:
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [60]:
df.fare.fillna(df.fare.mean(), inplace=True)
df.fare.isna().sum()

0

In [61]:
df.isna().sum()

pclass      0
survived    0
name        0
sex         0
age         0
sibsp       0
parch       0
ticket      0
fare        0
embarked    0
dtype: int64

In [62]:
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.000000,0,0,24160,211.3375,S
1,1,1,"Allison, Master. Hudson Trevor",male,0.916700,1,2,113781,151.5500,S
2,1,0,"Allison, Miss. Helen Loraine",female,2.000000,1,2,113781,151.5500,S
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.000000,1,2,113781,151.5500,S
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.000000,1,2,113781,151.5500,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.500000,1,0,2665,14.4542,C
1305,3,0,"Zabour, Miss. Thamine",female,29.881135,1,0,2665,14.4542,C
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.500000,0,0,2656,7.2250,C
1307,3,0,"Zakarian, Mr. Ortin",male,27.000000,0,0,2670,7.2250,C


In [63]:
df = df.drop(['name', 'ticket'], axis=1)
df

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.000000,0,0,211.3375,S
1,1,1,male,0.916700,1,2,151.5500,S
2,1,0,female,2.000000,1,2,151.5500,S
3,1,0,male,30.000000,1,2,151.5500,S
4,1,0,female,25.000000,1,2,151.5500,S
...,...,...,...,...,...,...,...,...
1304,3,0,female,14.500000,1,0,14.4542,C
1305,3,0,female,29.881135,1,0,14.4542,C
1306,3,0,male,26.500000,0,0,7.2250,C
1307,3,0,male,27.000000,0,0,7.2250,C


In [64]:
df.to_csv('titanic.csv')

In [65]:
from autoML import AutoML
automl = AutoML(df, y_colname='survived')
automl.getResults()

,algorithm,features,n_features,train_time,mem_max,f1,accuracy,roc_auc,confusion_matrix
0,LogisticRegression,"(pclass, sex_male)",2,1.618874,209.023438,0.743306,0.808999,0.851032,"[[140, 22], [28, 72]]"
1,SVC(probability=True),"(pclass, sex_male)",2,1.831457,210.023438,0.673333,0.786212,0.812746,"[[161, 1], [49, 51]]"
2,DecisionTreeClassifier,"(pclass, sex_male)",2,1.602379,210.640625,0.673333,0.786212,0.832358,"[[161, 1], [49, 51]]"
3,KNeighborsClassifier,"(pclass, sex_male)",2,1.629616,210.496094,0.543450,0.564804,0.703021,"[[161, 1], [49, 51]]"


In [66]:
# Mostrar as dimensões e as primerias linhas do dataset


In [67]:
# Mostra total de valores nulos por variável independente (features)


In [68]:
# Fazer um corte no dataset com as colunas:'age', 'pclass', 'sex', 'embarked', 'survived' e eliminar dados nulos.


In [69]:
# Mostrar novas dimensões do dataset após o corte


In [70]:
# Verifica balanceamento das classses (value_counts())


In [71]:
#mostra gráfico de barras do total de amostras por classe


## Pré-processamento

In [72]:
# Atribuir valores para X e y

# Mostrar as dimensões de X e y


### Encoding variáveis (features) categoricas :
Se as variáveis não possuem uma ordem específica, geralmente a melhor opção é one hot-encoding (dummy encoding)

In [73]:
# Cria X com as features codificadas com ".get_dummies" do Pandas.

# Mostra o resultado da codificação one-hot encoding


In [74]:
# Dividir dados em treino e teste (70/30)


## Treinamento dos modelos

### Knn - K vizinhos mais próximos (K Nearest neighbors)

In [75]:
# Importar o modelo knn Classifier


In [76]:
# Instanciar modelo de regressão logística


In [77]:
# Treinar o modelo com os dados de treinamento


In [78]:
# Verificar a acurácia do modelo (.score)


In [79]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


### Regressão Logística

In [80]:
# Instanciar modelo de regressão logística


In [81]:
# Treinar o modelo com os dados de treinamento


In [82]:
# Verificar a acurácia do modelo (.score)


In [83]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


## Avaliação da qualidade do melhor modelo

In [84]:
# Importar a biblioteca (plot_confusion_matrix)


In [85]:
# Mostrar a Matriz de Confusão do melhor modelo treinado


## Próximos passos
- Fazer normalização dos dados (Standard Scalar)   
- Treinar outros modelos:   
  - Árvore de decisão (Decision Tree Classifier)   
  - Floresta aleatória (Random Forest Classifier)

In [86]:
# Árvore de decisão
clf = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)
clf.score(X_test, y_test)

NameError: name 'DecisionTreeClassifier' is not defined

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_class = RandomForestClassifier(n_estimators=120, max_depth=6, random_state=100).fit(X_train, y_train)
rf_class.score(X_test, y_test)